In [1]:
!pip install --quiet climetlab


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


# Creating a shared dataset of GRIBs

In [2]:
import climetlab as cml

## Download data to the climetlab cache

In [3]:
for month in range(1, 13):  # This takes a few minutes.
    cml.load_source(
        "mars",
        param=["2t"],
        levtype="sfc",
        area=[50, -50, 20, 50],
        grid=[1, 1],
        date=f"2012-{month}",
    )

2022-12-06 14:12:29 ECMWF API python library 1.6.3
2022-12-06 14:12:29 ECMWF API at https://api.ecmwf.int/v1
2022-12-06 14:12:29 Welcome Florian Pinault
2022-12-06 14:12:30 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2022-12-06 14:12:30 Request submitted
2022-12-06 14:12:30 Request id: 638f3fbe457ff7b14dec36fc
2022-12-06 14:12:30 Request is submitted
2022-12-06 14:12:32 Request is active
2022-12-06 14:12:37 Calling 'nice mars /tmp/20221206-1310/23/tmp-_marsvHsjNS.req'
2022-12-06 14:12:37 mars - WARN -
2022-12-06 14:12:37 mars - WARN -
2022-12-06 14:12:37 MIR environment variables:
2022-12-06 14:12:37 MIR_CACHE_PATH=/data/ec_coeff
2022-12-06 14:12:37 Using MARS binary: /usr/local/apps/mars/versions/20220411095548/bin/mars.bin
2022-12-06 14:12:37 mars - INFO   - 20221206.131231 - Welcome to MARS
2022-12-06 14:12:37 mars - INFO   - 20221206.131231 - MARS Client build stamp: 20220411095548
2022-12-06 14:12:37 ma

In [4]:
cml.load_source(
    "mars",
    param="msl",
    levtype="sfc",
    area=[50, -50, 20, 50],
    grid=[1, 1],
    date="2012-12-01",
);

2022-12-06 14:14:24 ECMWF API python library 1.6.3
2022-12-06 14:14:24 ECMWF API at https://api.ecmwf.int/v1
2022-12-06 14:14:25 Welcome Florian Pinault
2022-12-06 14:14:25 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2022-12-06 14:14:26 Request submitted
2022-12-06 14:14:26 Request id: 638f40323677e48ef887c792
2022-12-06 14:14:26 Request is submitted
2022-12-06 14:14:27 Request is active
2022-12-06 14:14:32 Calling 'nice mars /tmp/20221206-1310/64/tmp-_marst1S1si.req'
2022-12-06 14:14:32 mars - WARN -
2022-12-06 14:14:32 mars - WARN -
2022-12-06 14:14:32 MIR environment variables:
2022-12-06 14:14:32 MIR_CACHE_PATH=/data/ec_coeff
2022-12-06 14:14:32 Using MARS binary: /usr/local/apps/mars/versions/20220411095548/bin/mars.bin
2022-12-06 14:14:32 mars - INFO   - 20221206.131426 - Welcome to MARS
2022-12-06 14:14:32 mars - INFO   - 20221206.131426 - MARS Client build stamp: 20220411095548
2022-12-06 14:14:32 ma

## Export the data to a shared directory

This is optional, you could keep working on the data from the cache if you are the only user of the data and you do not mind redownloading it later.
Other people should not use your cache:
- When using climetlab the cache will eventually fills up and the data may be deleted automatically,
- You will need to deal with permissions issues.
- It will make it difficult to share the data with other people.

Let us export the data to a shared directory `shared-data/temperature-for-analysis`

In [5]:
# Some housekeeping
!rm -rf shared-data/temperature-for-analysis
!mkdir -p shared-data/temperature-for-analysis

In [6]:
# export all data from my cache which is from mars and not older that 1 day
!climetlab export_cache shared-data/temperature-for-analysis --newer 1h --match mars

Copying cache entries matching 'mars' and newer than '2022-12-06 13:14:36' to shared-data/temperature-for-analysis.
100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 145.45it/s]
Copied 13 cache entries to shared-data/temperature-for-analysis.


## Create indexes to speed up data access when using it. (Optional)

In [7]:
!climetlab index_directory shared-data/temperature-for-analysis

  0%|                                                    | 0/13 [00:00<?, ?it/s]
Parsing shared-data/temperature-for-analysis/mars-retriever-ed2ab4d5cd22ea26a951
Parsing shared-data/temperature-for-analysis/mars-retriever-ed2ab4d5cd22ea26a951
  8%|███▍                                        | 1/13 [00:00<00:01,  7.06it/s]
Parsing shared-data/temperature-for-analysis/mars-retriever-4617cd8c5d7638680756
                                                                                
Parsing shared-data/temperature-for-analysis/mars-retriever-f7438015c79b0b04fd36
                                                                                
Parsing shared-data/temperature-for-analysis/mars-retriever-a151e7fcd829068805a0
                                                                                
Parsing shared-data/temperature-for-analysis/mars-retriever-753548dc22f2ee602d29
 38%|████████████████▉                           | 5/13 [00:00<00:00, 23.06it/s]
Parsing shared-data/temperat

In [8]:
!climetlab availability shared-data/temperature-for-analysis

## Using the data


In [9]:
DATA = "shared-data/temperature-for-analysis"

In [10]:
source = cml.load_source("directory", DATA)

In [11]:
source.availability

This is a good time to check the data, is all the data here? Are they missing dates? Parameters?

The data is ready to be used as numpy, tensorflow or xarray object.

In [12]:
source.sel(param="msl").to_numpy().mean()

101725.47522756307

In [13]:
cml.load_source("directory", DATA, param="msl").to_numpy().mean()

101725.47522756307

In [14]:
temp = source.sel(param="2t").order_by("date")
temp.to_tfdataset()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-06 14:14:54.269537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-06 14:14:54.269952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<ZipDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [15]:
temp.to_xarray()

<xarray.Dataset>
Dimensions:     (number: 1, time: 366, step: 1, surface: 1, latitude: 31,
                 longitude: 101)
Coordinates:
  * number      (number) int64 0
  * time        (time) datetime64[ns] 2012-01-01T12:00:00 ... 2012-12-31T12:0...
  * step        (step) timedelta64[ns] 00:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 50.0 49.0 48.0 47.0 ... 23.0 22.0 21.0 20.0
  * longitude   (longitude) float64 -50.0 -49.0 -48.0 -47.0 ... 48.0 49.0 50.0
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    t2m         (number, time, step, surface, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-12-06T14:14 GRIB to CDM+CF via cfgrib-0.9.1...

In [16]:
# Note that this is wrong (not implemented yet)
temp.availability